In [ ]:
# Mamy w szkole 6 nauczycieli i 3 klasy, które muszą mieć ułożony plan lekcji.
# Nauczyciele są z matmy, polskiego, historii, przyrody i angielskiego.
# Każda klasa musi mieć 5 matematyk, 5 polskich, 4 angielskie, 4 przyrody i 2 historie tygodniowo.
# Zajęcia muszą się odbyć do w godzinach 8 - 12:30 (łącznie maksymlanie 5 lekcji).
# Jak ułożyć plan dla tych 3 klas, by uczniowie i nauczyciele mieli jak najmniej okienek?


#
import random
from collections import defaultdict
from copy import deepcopy
from dataclasses import dataclass
from typing import List, Optional, Dict, Callable, Tuple

import numpy as np
from numpy import mean

N_PRZEDMIOTOW = 6
N_LEKCJI = 4
N_DNI_TYGODNIA = 5

@dataclass
class Dzien:
    przedmioty: List[int]
    def __init__(self, przedmioty: Optional[List[int]] = None):
        if przedmioty is not None:
            assert len(przedmioty) == N_LEKCJI
            for przedmiot in przedmioty:
                assert 0 <= przedmiot <= N_PRZEDMIOTOW
        else:
            przedmioty = list(np.random.randint(0, N_PRZEDMIOTOW+1, N_LEKCJI))
        self.przedmioty = przedmioty

@dataclass
class Klasa:
    dni: List[Dzien]
    oczekiwane_liczby_przedmiotow: Dict[int, int]

    def __init__(self, oczekiwane_liczby_przedmiotow: Dict[int, int], dni: Optional[List[Dzien]] = None):
        self.oczekiwane_liczby_przedmiotow = oczekiwane_liczby_przedmiotow
        if dni is not None:
            assert len(dni) == N_DNI_TYGODNIA
            for dzien in dni:
                assert len(dzien.przedmioty) == N_LEKCJI
                for przedmiot in dzien.przedmioty:
                    assert 0 <= przedmiot <= N_PRZEDMIOTOW
        else:
            dni = list([Dzien() for i in range(N_DNI_TYGODNIA)])
        self.dni = dni

    def oblicz_liczby_przedmiotow(self) -> Dict[int, int]:
        liczby_przedmiotow = {i: 0 for i in range(N_PRZEDMIOTOW+1)}
        for dzien in self.dni:
            for przedmiot in dzien.przedmioty:
                liczby_przedmiotow[przedmiot] += 1
        return liczby_przedmiotow

    def ile_przedmiotow_zlych(self) -> int:
        suma_roznic = 0
        liczby_przedmiotow = self.oblicz_liczby_przedmiotow()
        for id, liczba_przedmiotow in liczby_przedmiotow.items():
            if id != 0:
                roznica = liczba_przedmiotow - self.oczekiwane_liczby_przedmiotow[id]
                suma_roznic += abs(roznica)
        return suma_roznic

    def ile_okienek(self) -> int:
        liczba_okienek = 0
        for dzien in self.dni:
            przedmioty = list(dzien.przedmioty)
            while len(przedmioty) > 0 and przedmioty[0] == 0:
                przedmioty = przedmioty[1:]
            while len(przedmioty) > 0 and przedmioty[-1] == 0:
                przedmioty = przedmioty[:-1]
            liczba_okienek += np.sum(np.array(przedmioty) == 0)
        return liczba_okienek


liczba_klas = 3
oczekiwane_liczby_przedmiotow = {
    1: 4, #matma
    2: 4, #polski
    3: 3, #angielski
    4: 3, #przyroda
    5: 2, #historia
    6: 2, #wf
}
przedmiot_do_nazwy = {
    0: "",
    1: "matematyka",
    2: "polski",
    3: "angielski",
    4: "przyroda",
    5: "historia",
    6: "wf",
}


@dataclass
class Plan:
    klasy: List[Klasa]
    kara: int = 1000
    liczby: Tuple[int, int, int, int] = 0, 0, 0, 0
    kara_za_powtorzenia: int = 10
    kara_za_nadmierne_przedmioty: int = 10
    kara_za_okienka_klas: int = 2
    kara_za_okienka_nauczycieli: int = 1

    def __init__(self, klasy: List[Klasa]):
        self.klasy = klasy

    def ile_przedmiotow_takich_samych(self) -> int:
        powtorzenia_przedmiotow = 0
        for dzien in range(N_DNI_TYGODNIA):
            for lekcja in range(N_LEKCJI):
                przedmioty = defaultdict(lambda: 0)
                for klasa in self.klasy:
                    przedmioty[klasa.dni[dzien].przedmioty[lekcja]] += 1
                for id, zliczenie in przedmioty.items():
                    if id != 0:
                        if zliczenie > 1:
                            powtorzenia_przedmiotow += (zliczenie - 1)
        return powtorzenia_przedmiotow

    def ile_okienek_nauczycieli(self) -> int:
        liczba_okienek = 0
        for dzien in range(N_DNI_TYGODNIA):
            przedmioty_nauczycieli = defaultdict(lambda: [False for i in range(N_LEKCJI)])
            for lekcja in range(N_LEKCJI):
                for klasa in self.klasy:
                    przedmioty_nauczycieli[klasa.dni[dzien].przedmioty[lekcja]][lekcja] = True
            if 0 in przedmioty_nauczycieli:
                przedmioty_nauczycieli.pop(0)
            for przedmiot, godziny in przedmioty_nauczycieli.items():
                przedmioty = list(godziny)
                while len(przedmioty) > 0 and przedmioty[0] == False:
                    przedmioty = przedmioty[1:]
                while len(przedmioty) > 0 and przedmioty[-1] == False:
                    przedmioty = przedmioty[:-1]
                liczba_okienek += np.sum(np.array(przedmioty) == False)
        return liczba_okienek

    def ewaluuj(self):
        powtorzenia_przedmiotow_nauczycieli = self.ile_przedmiotow_takich_samych()
        liczba_okienek_nauczycieli = self.ile_okienek_nauczycieli()
        przedmiotow_zlych = 0
        liczba_okienek_uczniow = 0
        for klasa in self.klasy:
            przedmiotow_zlych += klasa.ile_przedmiotow_zlych()
            liczba_okienek_uczniow += klasa.ile_okienek()

        self.liczby = (
            powtorzenia_przedmiotow_nauczycieli,
            przedmiotow_zlych,
            liczba_okienek_uczniow,
            liczba_okienek_nauczycieli
        )

        self.kara = (
            self.kara_za_powtorzenia * powtorzenia_przedmiotow_nauczycieli +
            self.kara_za_nadmierne_przedmioty * przedmiotow_zlych +
            self.kara_za_okienka_klas * liczba_okienek_uczniow +
            self.kara_za_okienka_nauczycieli * liczba_okienek_nauczycieli
        )

    def pobierz_kary_ladnie(self):
        return f"Kara {self.kara}, jeden nauczyciel dwie lekcje naraz {self.liczby[0]}, " \
               f"za duzo przedmiotow {self.liczby[1]}, okienka uczniow {self.liczby[2]}," \
               f" okienka nauczycieli {self.liczby[3]}."

    def wypisz_plan(self):
        for i, klasa in enumerate(self.klasy):
            print(f"\n Klasa {i+1}")
            print(f"| godzina | poniedzialek |   wtorek   |    sroda   |  czwartek  |   piatek   |")
            print("_______________________________________________________________________________")
            for godzina in range(N_LEKCJI):
                print('| {:^7} | {:^12} | {:^10} | {:^10} | {:^10} | {:^10} |'.format(
                *[godzina + 8] + [
                    przedmiot_do_nazwy[dzien.przedmioty[godzina]] for dzien in klasa.dni
                ]))


def tworz_populacje(wielkosc):
    plany = []
    for i in range(wielkosc):
        klasy = []
        for j in range(liczba_klas):
            klasy.append(Klasa(oczekiwane_liczby_przedmiotow))
        plany.append(Plan(klasy))
    return plany

def mutacja_osobnika(osobnik: Plan, opcje):
    klasy = osobnik.klasy
    klasy[random.randint(0, liczba_klas-1)].dni[random.randint(0, N_DNI_TYGODNIA-1)].przedmioty[random.randint(0, N_LEKCJI-1)] = random.randint(0, N_PRZEDMIOTOW)
    return Plan(klasy)

def mutacja_populacji(populacja, pstwo_mutacji=0.4, funkcja_mutacji=mutacja_osobnika, opcje: Optional[List[int]] = None):
    zmutowani_osobnicy = [
        funkcja_mutacji(deepcopy(osobnik), opcje) for osobnik in populacja if random.random() < pstwo_mutacji
    ]
    return populacja + zmutowani_osobnicy

def krzyzuj_rodzicow(rodzic1, rodzic2):
    klasy = rodzic1.klasy
    losowa_klasa = random.randint(0, liczba_klas-1)
    klasy[losowa_klasa] = deepcopy(rodzic2.klasy[losowa_klasa])
    return Plan(klasy)

def _szukaj_drugiego_rodzica(populacja, i):
    losowy_index = random.randint(0, len(populacja) - 2)
    if losowy_index >= i:
        losowy_index += 1
    return populacja[losowy_index]

def krzyzowanie_populacji(populacja, pstwo_krzyzowania=0.4, funkcja_krzyzowania=krzyzuj_rodzicow):
    krzyzowani_osobnicy = [
        funkcja_krzyzowania(deepcopy(osobnik), deepcopy(_szukaj_drugiego_rodzica(populacja, i)))
        for i, osobnik in enumerate(populacja)
        if random.random() < pstwo_krzyzowania
    ]
    return populacja + krzyzowani_osobnicy

def selekcja_rankingowa(populacja, wielkosc_po_selekcji=100):
    wyniki = np.array([-osobnik.kara for osobnik in populacja])
    indeksy_wynikow = np.argsort(wyniki)[::-1]
    return list(np.array(populacja)[indeksy_wynikow[:wielkosc_po_selekcji]])

def selekcja_turniejowa(populacja, wielkosc_po_selekcji=100):
    wyniki = np.array([osobnik.kara for osobnik in populacja])
    standaryzuj = max(max(wyniki) + 10, 100)
    wyniki = standaryzuj - wyniki
    kumulatywne_sumy = np.cumsum(wyniki)
    losowe_liczby = np.random.random(wielkosc_po_selekcji-5) * kumulatywne_sumy[-1]

    indeksy_wynikow = np.argsort(wyniki)[::-1]
    nowa_populacja = list(np.array(populacja)[indeksy_wynikow[:5]])
    for losowa_liczba in losowe_liczby:
        indeks = 0
        while losowa_liczba > kumulatywne_sumy[indeks]:
            indeks += 1
            if indeks == len(populacja):
                print(losowa_liczba), print(kumulatywne_sumy)
        nowa_populacja.append(deepcopy(populacja[indeks]))

    return nowa_populacja

def mutacja_osobnika(osobnik: Plan, opcje: Optional[List[int]] = None):
    if opcje == None:
        opcje = [3]
    opcja = np.random.choice(opcje, 1)

    klasy = osobnik.klasy
    losowa_klasa = random.randint(0, liczba_klas-1)
    losowa_klasa2 = random.randint(0, liczba_klas-1)
    losowy_dzien = random.randint(0, N_DNI_TYGODNIA-1)
    losowy_dzien2 = random.randint(0, N_DNI_TYGODNIA-1)
    losowa_lekcja = random.randint(0, N_LEKCJI-1)
    losowa_lekcja2 = random.randint(0, N_LEKCJI-1)

    nowy_losowy_przedmiot = random.randint(0, N_PRZEDMIOTOW)
    nowy_losowy_dzien = Dzien()
    nowa_losowa_klasa = Klasa(oczekiwane_liczby_przedmiotow)

    # losowa lekcja jest wybierana na nowo
    if opcja == 1:
        klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja] = nowy_losowy_przedmiot
    # losowy dzien jest wybierany na nowo
    elif opcja == 2:
        klasy[losowa_klasa].dni[losowy_dzien] = nowy_losowy_dzien
    # losowa klasa otrzymuje nowy plan
    elif opcja == 3:
        klasy[losowa_klasa] = nowa_losowa_klasa
    # 2 losowe klasy są zmieniane
    elif opcja == 4:
        klasy[losowa_klasa], klasy[losowa_klasa2] = klasy[losowa_klasa2], klasy[losowa_klasa]
    # 2 losowe dni są zmieniane
    elif opcja == 5:
        klasy[losowa_klasa].dni[losowy_dzien], klasy[losowa_klasa2].dni[losowy_dzien2] = (
            klasy[losowa_klasa2].dni[losowy_dzien2], klasy[losowa_klasa].dni[losowy_dzien]
        )
    # 2 losowe dni w jednej klasie są zmieniane
    elif opcja == 6:
        klasy[losowa_klasa].dni[losowy_dzien], klasy[losowa_klasa].dni[losowy_dzien2] = (
            klasy[losowa_klasa].dni[losowy_dzien2], klasy[losowa_klasa].dni[losowy_dzien]
        )
    # 2 losowe lekcje w różnych klasie są zmieniane
    elif opcja == 7:
        klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja], klasy[losowa_klasa2].dni[losowy_dzien2].przedmioty[losowa_lekcja2] = (
            klasy[losowa_klasa2].dni[losowy_dzien2].przedmioty[losowa_lekcja2], klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja]
        )
    # 2 losowe lekcje w jednej klasie są zmieniane
    elif opcja == 8:
        klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja], klasy[losowa_klasa].dni[losowy_dzien2].przedmioty[losowa_lekcja2] = (
            klasy[losowa_klasa].dni[losowy_dzien2].przedmioty[losowa_lekcja2], klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja]
        )
    # 2 losowe lekcje tego samego dnia w jednej klasie są zmieniane
    elif opcja == 9:
        klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja], klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja2] = (
            klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja2], klasy[losowa_klasa].dni[losowy_dzien].przedmioty[losowa_lekcja]
        )

    return Plan(klasy)

def krzyzuj_rodzicow(rodzic1, rodzic2):
    klasy = rodzic1.klasy
    losowa_klasa = random.randint(0, liczba_klas-1)
    klasy[losowa_klasa] = deepcopy(rodzic2.klasy[losowa_klasa])
    return Plan(klasy)

pstwo_krzyzowania = 0.3
pstwo_mutacji = 0.5
opcje_mutacji = [1, 2]
selekcja = 2  # 1 - rankingowa; 2 - turniejowa
wielkosc_populacji = 100
liczba_generacji = 1

polowa_populacji = int(wielkosc_populacji / 2)
populacja = tworz_populacje(wielkosc=wielkosc_populacji)
for i in range(liczba_generacji):
    [osobnik.ewaluuj() for osobnik in populacja]
    if selekcja == 1:
        populacja = selekcja_rankingowa(populacja, polowa_populacji)
    else:
        populacja = selekcja_turniejowa(populacja, polowa_populacji)
    wynik = populacja[0].kara
    sredni_wynik = mean([osobnik.kara for osobnik in populacja[:polowa_populacji]])
    print(populacja[0].pobierz_kary_ladnie(), ", sredni wynik ", sredni_wynik)
    populacja = mutacja_populacji(populacja, pstwo_mutacji, mutacja_osobnika, opcje_mutacji)
    populacja = krzyzowanie_populacji(populacja, pstwo_krzyzowania, krzyzuj_rodzicow)
populacja[0].wypisz_plan()


from copy import copy
from random import randint, random

import numpy as np
from numpy import mean

SECRET_PASSWORD = "4729210302"
N_CHROMOSOMES = 2


# Chcemy uzyskac haslo do systemu. Wiemy ze haslo sklada sie z 10 cyfr, ale jakie to cyfry?
# Mozemy sprobowac znalezc to haslo w sposob losowy, np. wypisujac kolejno 0123456789, 0123456790, ...
# Ale mozemy wykorzystac tez algorytmy genetyczne.

# Algorytm genetyczny opiera sie na ulepszaniu populacji. Populacja sklada sie z osobnikow, a osobnik ma swoj genom.
# W kazdym zastosowaniu nalezy zdefiniowac wiele rzeczy. Zacznijmy po kolei.

# 1. Populacja
# Niech pojedynczym osobnikiem bedzie potencjalne haslo. Zatem jeden osobnik to na przyklad 0123456789.
# Kazdy osobnik sklada sie z genow. Tutaj geny to sa cyfry. Zatem kazdy osobnik ma 10 genow, ktore sa cyframi.
# Czy kolejnosc genow ma znaczenie?
# Tutaj ma, przeciez osobnicy zlozeni dokladnie z tych samych genow 0123456789 i 0123456798 reprezentuja rozne liczby.
# Ale nie zawsze kolejnosc musi miec znaczenie.

# Podczas inicjalizacji populacji powinnismy stworzyc kilka/kilkanascie/dziesiat osobnikow o roznym genomie.
# Zacznijmy od 20 poczatkowych losowych osobnikow.
# inicjalizacja populacji
def get_random_individual(n_chromosomes=10):
    assert n_chromosomes > 0, "Number of chromosomes should be greater than 0."
    individual = ""
    for chromosome in range(n_chromosomes):
        individual = individual + str(randint(0, 9))
    return individual


def initialize_population(size=10, n_chromosomes=10):
    return [
        get_random_individual(n_chromosomes) for i in range(size)
    ]


population = initialize_population(size=20, n_chromosomes=N_CHROMOSOMES)
print(population)


# 2 Selekcja
# Mamy jakies losowe hasla. Ale po co nam losowe hasla? Musimy umiec oceniac ich przydatnosc.
# Tak sie sklada, ze system podczas logowania mowi nam ile cyfr hasla jest poprawnych.
def score_individual(individual):
    score = 0
    for individual_chromosome, expected_chromosome in zip(individual, SECRET_PASSWORD):
        if individual_chromosome == expected_chromosome:
            score += 1
    return score


def select_population(population, expected_size=10):
    scores = np.array([score_individual(individual) for individual in population])
    scores_indices = np.argsort(scores)[::-1]
    return list(np.array(population)[scores_indices[:expected_size]])


selected_population = select_population(population)
print(selected_population)


# 3 Mutacja
# W jakis sposob powinnismy zaczac sie zblizac na poprawnego hasla. Mozemy to robic poprzez
# zmiany w obecnym hasle. Dlaczego to moze dzialac?
# Poniewaz robimy selekcje i wybieramy te hasla ktore sa blizej poprawnego, zatem edytujemy hasla blizsze poprawnemu.


def mutate_one(individual):
    individual = [gene for gene in individual]
    individual[randint(0, len(individual)-1)] = str(randint(0, 9))
    return ''.join(individual)


def mutate(population, mutation_p=0.4):
    mutated = [
        mutate_one(individual) for individual in population if random() < mutation_p
    ]
    return population + mutated


mutated_population = mutate(selected_population)
print(mutated_population)


# Sprawdzmy jak dziala nasz algorytm z tym momencie.
# Uruchomimy go na 100 generacji.

N_GENERATIONS = 1
MUTATION_P = 0.4

population = initialize_population(size=20, n_chromosomes=10)
for generation in range(N_GENERATIONS):
    selected_population = select_population(population, 10)
    print(f"Generation {generation+1}, best score {score_individual(selected_population[0])} for password {selected_population[0]}.")
    population = mutate(selected_population, MUTATION_P)


# WOW! Udalo sie znalezc haslo! Ile prób zgadniecia hasla mielismy? Jak to obliczyc?
# 4. Krzyzowanie
# Ok, ale mamy jeszcze jednego agenta w kieszeni. Otoz w biologii osobniki sie rozmnazaja.
# Podczas rozmnazania potomkowie otrzymuja geny od obu rodzicow, czyli sa usrednieniem rodzicow.
# Moze sie okazac, ze od obu rodzicow dostana geny bardziej przydatne lub od obu mniej przydatne albo mieszanke.
# U nas rodzicami sa hasla i potomkami tez. Jak zrobic krzyzowanie 2 hasel?
# Wezmy pierwsza polowe hasla od pierwszego rodzica, a druga polowe od drugiego rodzica.

def cross_one(first_parent, second_parent):
    index = int(len(first_parent)/2)
    # index = randint(0, len(first_parent) - 1)
    return first_parent[:index] + second_parent[index:]


def _get_other_parent(population, i):
    random_index = randint(0, len(population) - 2)
    if random_index >= i:
        random_index += 1
    return population[random_index]


def cross(population, cross_p=0.4):
    crossed = [
        cross_one(individual, _get_other_parent(population, i))
        for i, individual in enumerate(population)
        if random() < cross_p
    ]
    return population + crossed


# Sprawdzmy jak teraz dziala nasz algorytm.
# Uruchomimy go na 100 generacjach, ale z krzyzowaniem.

N_TRIES = 1
MUTATION_P = 0.4
CROSS_P = 0.4

generations = []
for trying in range(N_TRIES):
    score = 0
    generation = 0
    population = initialize_population(size=20, n_chromosomes=10)
    while score<10:
        generation += 1
        population = select_population(population, 10)
        score = score_individual(population[0])
        # print(f"Generation {generation+1}, best score {score_individual(population[0])} for password {population[0]}.")
        population = mutate(population, MUTATION_P)
        population = cross(population, CROSS_P)
    generations.append(generation)

print(f"Mean generation {mean(generations)}.")

# Okazuje sie ze algorytm zbiegl wolniej. Ile tym razem zostalo utworzonych hasel?
# Czy potraficie zmienic cos w tym algorytmie, by dzialal szybciej?
# Czy mozna jakos zmienic krzyzowanie, mutacje?
